# 海洋要素计算 编程作业4
计算地转流  
- 1.利用两月的月平均资料计算北太平洋*6°N-35°N*范围内的地转流，选取**1500db**作为参考零面画出*10db*，*100 db*，*250 db*，*500 db*等四个深度层上的流场和流速
- 2.利用上面计算结果，计算北赤道流水体输运（如130°E，8°N-18°N断面），比较讨论两月结果差异，也可进一步比较不同断面的差异  
  
**Developed By [Hanxue Yu](https://github.com/Yuhan-xue) 02\06\2023**  
**Student ID: 20010006082**

## Import Module

In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gsw

## NC file Read&Load

### Read

In [2]:
# NC 文件加载与信息提取
das1=nc.Dataset('TS_201801_GLB.nc')
das2=nc.Dataset('TS_201807_GLB.nc')
def getNCinfo(das):
    for i in list(das.variables.keys()):
        ignolist=['time','lat','lon','LONGITUDE', 'LATITUDE', 'TIME', 'bnds','err','ERR','error','ERROR']
        if i in ignolist or i.split('_')[-1] in ignolist:
            continue
        print(f'Key: {i}')
        print(f'    | Long Name: {das.variables[i].long_name}')
        print(f'    | Units: {das.variables[i].units}')
        print(f'    | Shape: {das.variables[i].shape}')
getNCinfo(das1)

Key: PRES
    | Long Name: Pressure
    | Units: decibar
    | Shape: (25,)
Key: TOI
    | Long Name: Temperature.(ITS90)
    | Units: degree_Celsius
    | Shape: (25, 132, 360)
Key: SOI
    | Long Name: Salinity.(PSS-78)
    | Units: psu
    | Shape: (25, 132, 360)


### Load

In [3]:
# 变量加载
## 读取
DB_01=np.array(das1.variables['PRES'])
TOI_01=np.array(das1.variables['TOI'])
SOI_01=np.array(das1.variables['TOI'])
DB_07=np.array(das2.variables['PRES'])
TOI_07=np.array(das2.variables['TOI'])
SOI_07=np.array(das2.variables['TOI'])
lon=np.array(das1.variables['LONGITUDE'])
lat=np.array(das1.variables['LATITUDE'])
## 处理
lon[lon<0]=lon[lon<0]+360
SOI_01[SOI_01>1000]=np.nan
TOI_01[TOI_01>1000]=np.nan
SOI_07[SOI_07>1000]=np.nan
TOI_07[TOI_07>1000]=np.nan
DB_01=np.broadcast_to(DB_01[:,np.newaxis,np.newaxis],SOI_01.shape)
DB_01=np.broadcast_to(DB_07[:,np.newaxis,np.newaxis],SOI_07.shape)

## Calculate Lattice Distance and Coriolis Parameters

In [13]:
# 计算格点距离和科氏参数
def getdx(lon):
    dx=np.zeros(len(lon))
    dx[:-1]=111*(lon[1:]-lon[:-1])
    return np.abs(dx[:-1])
def getdy(lat):
    dy=np.zeros(len(lat))
    dy[:-1]=111*(lat[1:]-lat[:-1])*np.cos(lat[:-1]*np.pi/180)
    return np.abs(dy[:-1])
def getCoriolisParm(lat):
    return 2 * 7.29e-5 * np.sin(np.radians(lat))
dx,dy,f=getdx(lon),getdy(lat),getCoriolisParm(lat)

## Calculates specific volume anomaly
查询[GSW-Python](https://teos-10.github.io/GSW-Python/gsw_flat.html)文档中gsw.specvol_anom_standard描述与用法
- 描述
    - Calculates specific volume anomaly from Absolute Salinity, Conservative Temperature and pressure. It uses the computationally-efficient expression for specific volume as a function of SA, CT and p (Roquet et al., 2015). The reference value to which the anomaly is calculated has an Absolute Salinity of SSO and Conservative Temperature equal to 0 degrees C. 
- 输入参数  
    - SA : Absolute Salinity, g/kg  
    - CT : Conservative Temperature (ITS-90), degrees C  
    - p : Sea pressure (absolute pressure minus 10.1325 dbar), dbar  

In [8]:
# 计算比容异常<使用了GSW-Python模块>
anomaly = gsw.specvol_anom_standard(SA=SOI_01, CT=TOI_01, p=DB_01)

In [ ]:
RefLevel=1500